In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from itertools import chain

import pandas as pd
import numpy as np
import time
import random


import re

In [2]:
# 这里的cookie换成你自己的cookie
cookies = {
    'JSESSIONID': 'F53385626720Dxxxxxxx',
    '_pk_id.7.eda7': 'fbfbdec1aa1b4c42.163xxxxx.',
}

headers = {
    'Connection': 'keep-alive',
    'Accept': 'application/xml, text/xml, */*; q=0.01',
    'X-Requested-With': 'XMLHttpRequest',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Origin': 'http://www.iachina.cn',
    'Referer': 'http://www.iachina.cn/col/col41/index.html?uid=1508&pageNum=7',
    'Accept-Language': 'zh-CN,zh;q=0.9',
}

params = (
    ('startrecord', '1'),
    ('endrecord', '140'),
    ('perpage', '108'),
)

data = {
  'col': '1',
  'appid': '1',
  'webid': '1',
  'path': '/',
  'columnid': '41',
  'sourceContentType': '8',
  'unitid': '1508',
  'webname': '\u4E2D\u56FD\u4FDD\u9669\u884C\u4E1A\u534F\u4F1A',
  'permissiontype': '0'
}

response = requests.post('http://www.iachina.cn/module/web/jpage/dataproxy.jsp', headers=headers, params=params, cookies=cookies, data=data, verify=False)
soup = BeautifulSoup(response.text, 'lxml')
chapter_list = soup.find_all(name='a')
all_chapter_url = [f"http://www.iachina.cn{i.get('href')}" for i in chapter_list]
all_chapter_text = [i.text for i in chapter_list]


In [10]:
pd.DataFrame({'url':all_chapter_url,
              'text':all_chapter_text})

,url,text
0,http://www.iachina.cn/art/2018/4/3/art_617_102...,2018年1-2月保险业经营情况表
1,http://www.iachina.cn/art/2018/3/9/art_617_102...,2018年1月保险业经营情况表
2,http://www.iachina.cn/art/2018/3/6/art_617_102...,2017年保险业经营情况表
3,http://www.iachina.cn/art/2018/1/10/art_617_10...,2017年1-11月保险业经营情况表
4,http://www.iachina.cn/art/2017/11/27/art_617_9...,2017年1-10月保险业经营情况表
...,...,...
102,http://www.iachina.cn/art/2009/11/5/art_617_11...,2009年1-5月保险业经营数据
103,http://www.iachina.cn/art/2009/11/5/art_617_11...,2009年1-4月保险业经营数据
104,http://www.iachina.cn/art/2009/11/5/art_617_11...,2009年1-3月保险业经营数据
105,http://www.iachina.cn/art/2009/11/5/art_617_11...,2009年1-2月保险业经营数据


In [8]:
def clean_column(columns_list):
    """
    这个代码负责将表格的列名称清洗干净（就是将一些二级标题、三级标题都转换成完整的名称：{一级标题}-{二级标题}-{三级标题})
    如果标题是一级标题 -> 一级标题
    如果标题是二级标题 -> {一级标题}-{二级标题}
    如果标题是三级标题 -> {一级标题}-{二级标题}-{三级标题}

    这么一做，每一个表的列名称就不会混淆

    """

    column_data = pd.DataFrame({'column_name':[i.replace(" ", '') for i in columns_list]})
    # 做一级填充
    column_data['class_1'] = column_data['column_name'].apply(lambda x: len(re.findall(pattern='[0-9]+、', string=x)) !=1)#[len(re.findall(pattern='[0-9]+、', string=temp_column)) !=1 for temp_column in columns_list]
    column_data['class_1_name'] = column_data['column_name']
    column_data.loc[column_data['class_1']== False, 'class_1_name'] = np.nan
    column_data['class_1_name'] = column_data['class_1_name'].ffill()
    # column_data.loc[column_data['column_name'] != column_data['class_1_name'],'class_1_name'] = \
    #     column_data.loc[column_data['column_name'] != column_data['class_1_name'],:].apply(lambda x: f"{x['class_1_name']}__{x['column_name']}", axis=1)
    column_data.loc[column_data['class_1'] == False,'class_1_name'] = \
        column_data.loc[column_data['class_1'] == False,:].apply(lambda x: f"{x['class_1_name']}__{x['column_name']}", axis=1)
    # 做二级填充

    column_data['class_2'] = column_data['column_name'].apply(lambda x: len(re.findall(pattern='（[0-9]+）', string=x))!=1)

    column_data['class_2_name'] = column_data['class_1_name']
    column_data.loc[column_data['class_2'] == False, 'class_2_name'] = np.nan
    column_data['class_2_name'] = column_data['class_2_name'].ffill()

    column_data.loc[column_data['class_2'] == False, 'class_2_name'] = column_data.loc[column_data['class_2'] == False, :].apply(lambda x: f"{x['class_2_name']}__{x['column_name']}", axis=1)

    return column_data['class_2_name'].tolist()



def get_chapter_data(index):
    # time.sleep(random.randint(0, 1.5))

    temp_url = all_chapter_url[index]
    temp_info = all_chapter_text[index]
    data = pd.read_html(temp_url)[2].dropna()
    data = data.loc[data[0] != data[1], :]
    data = data.set_index(0).T.reset_index(drop=True)

    columns_list = data.columns.tolist()
    data.columns = clean_column(columns_list)

    # data.columns = [f"{index}_{columns_list[index]}" for index in range(len(columns_list))]


    data['from_url'] = temp_url
    data['info'] = temp_info
    return data

# test function
test_data = get_chapter_data(index=70)
test_data

,原保险保费收入,1 、财产险,2 、人身险,（ 1 ）寿险,（ 2 ）健康险,（ 3 ）人身意外伤害险,养老保险公司企业年金缴费,原保险赔付支出,1 、财产险,2 、人身险,...,（ 2 ）健康险,（ 3 ）人身意外伤害险,业务及管理费,银行存款,投资,资产总额,养老保险公司企业年金受托管理资产,养老保险公司企业年金投资管理资产,from_url,info
0,48348140.79,13004974.81,35343165.99,32028411.67,2289052.69,1025701.62,2067390.99,11719708.42,5726104.19,5993604.23,...,729704.47,224487.94,4672660.26,206984823.51,381839223.50,636786170.04,15614184.12,14523965.45,http://www.iachina.cn/art/2012/5/4/art_617_110...,2012年1-3月保险业经营情况表


In [5]:
finaldata = pd.concat([get_chapter_data(index) for index in tqdm(range(len(all_chapter_url)))]).reset_index(drop=True)
finaldata.head(10)

100%|██████████| 107/107 [00:25<00:00,  4.21it/s]


,原保险保费收入,原保险保费收入__1、财产险,原保险保费收入__2、人身险,原保险保费收入__2、人身险__（1）寿险,原保险保费收入__2、人身险__（2）健康险,原保险保费收入__2、人身险__（3）人身意外伤害险,人身保险公司保户投资款新增交费,人身保险公司投连险独立账户新增交费,原保险赔付支出,原保险赔付支出__1、财产险,...,原保险赔付支出__2、人身险__（3）人身意外伤害险,业务及管理费,银行存款,投资,资产总额,from_url,info,养老保险公司企业年金缴费,养老保险公司企业年金受托管理资产,养老保险公司企业年金投资管理资产
0,97034372.62,17721912.27,79312460.35,69017036.90,8749972.64,1545450.80,25134457.09,1509510.17,22556676.55,8241803.44,...,408911.78,6151768.99,190830329.04,1309099035.19,1688522210.14,http://www.iachina.cn/art/2018/4/3/art_617_102...,2018年1-2月保险业经营情况表,NaN,NaN,NaN
1,68519191.78,11454663.02,57064528.75,50787454.05,5323376.13,953698.58,18563165.10,869157.37,13131087.35,4753829.05,...,231357.34,3179866.72,188340380.65,1313660366.91,1690528099.26,http://www.iachina.cn/art/2018/3/9/art_617_102...,2018年1月保险业经营情况表,NaN,NaN,NaN
2,365810073.85,98346579.05,267463494.80,214555650.29,43894603.83,9013240.68,58923639.73,4704175.97,111807932.57,50874495.97,...,2236859.45,42880643.40,192740725.29,1299321408.14,1674893735.07,http://www.iachina.cn/art/2018/3/6/art_617_102...,2017年保险业经营情况表,NaN,NaN,NaN
3,343975833.44,88130729.56,255845103.88,206441403.23,41055434.91,8348265.74,55361017.40,4157640.32,100908462.55,44522715.32,...,1994767.88,37058547.65,193021087.40,1277177694.44,1664096886.61,http://www.iachina.cn/art/2018/1/10/art_617_10...,2017年1-11月保险业经营情况表,NaN,NaN,NaN
4,323887679.96,79611362.01,244276317.95,198156253.23,38420882.44,7699182.28,50765401.89,3972071.35,90846751.60,39443910.98,...,1784671.48,33410032.12,191123741.21,1278381459.75,1661984320.20,http://www.iachina.cn/art/2017/11/27/art_617_9...,2017年1-10月保险业经营情况表,NaN,NaN,NaN
5,304573173.18,72302834.31,232270338.87,189409170.97,35830499.74,7030668.16,46524958.71,3806503.17,83272656.90,35669905.50,...,1623561.88,30425390.17,201134669.56,1263582594.79,1658243165.53,http://www.iachina.cn/art/2017/11/21/art_617_9...,2017年1-9月保险业经营情况表,NaN,NaN,NaN
6,276541769.99,63881132.88,212660637.11,174654969.13,31949255.13,6056412.84,41739970.35,3162867.33,74363643.40,31246191.59,...,1413369.23,26731367.16,196178261.80,1250080800.81,1636143978.33,http://www.iachina.cn/art/2017/10/9/art_617_95...,2017年1-8月保险业经营情况表,NaN,NaN,NaN
7,252675555.16,56292689.59,196382865.57,162214613.76,28864361.71,5303890.11,38097394.40,2395478.90,65844323.44,26970622.18,...,1202054.73,23567819.20,195881169.66,1246764619.36,1632382960.40,http://www.iachina.cn/art/2017/9/1/art_617_942...,2017年1-7月保险业经营情况表,NaN,NaN,NaN
8,231401495.45,48524378.57,182877116.88,152096140.18,26156889.00,4624087.69,34828860.06,1646921.93,57855012.90,22945019.64,...,1017501.38,20341100.70,215932308.16,1234056354.15,1643044791.18,http://www.iachina.cn/art/2017/8/10/art_617_94...,2017年1-6月保险业经营情况表,NaN,NaN,NaN
9,202918007.55,39564149.14,163353858.41,137173851.74,22400011.13,3779995.54,30843720.02,1213315.76,49001009.68,18846693.75,...,838250.71,16545616.74,234049445.24,1190286076.37,1606259671.62,http://www.iachina.cn/art/2017/6/28/art_617_94...,2017年1-5月保险业经营情况表,NaN,NaN,NaN


In [17]:
finaldata.to_csv("保险业经营数据.csv", index=False)

In [9]:
finaldata

,原保险保费收入,原保险保费收入__1、财产险,原保险保费收入__2、人身险,原保险保费收入__2、人身险__（1）寿险,原保险保费收入__2、人身险__（2）健康险,原保险保费收入__2、人身险__（3）人身意外伤害险,人身保险公司保户投资款新增交费,人身保险公司投连险独立账户新增交费,原保险赔付支出,原保险赔付支出__1、财产险,...,原保险赔付支出__2、人身险__（3）人身意外伤害险,业务及管理费,银行存款,投资,资产总额,from_url,info,养老保险公司企业年金缴费,养老保险公司企业年金受托管理资产,养老保险公司企业年金投资管理资产
0,97034372.62,17721912.27,79312460.35,69017036.90,8749972.64,1545450.80,25134457.09,1509510.17,22556676.55,8241803.44,...,408911.78,6151768.99,190830329.04,1309099035.19,1688522210.14,http://www.iachina.cn/art/2018/4/3/art_617_102...,2018年1-2月保险业经营情况表,NaN,NaN,NaN
1,68519191.78,11454663.02,57064528.75,50787454.05,5323376.13,953698.58,18563165.10,869157.37,13131087.35,4753829.05,...,231357.34,3179866.72,188340380.65,1313660366.91,1690528099.26,http://www.iachina.cn/art/2018/3/9/art_617_102...,2018年1月保险业经营情况表,NaN,NaN,NaN
2,365810073.85,98346579.05,267463494.80,214555650.29,43894603.83,9013240.68,58923639.73,4704175.97,111807932.57,50874495.97,...,2236859.45,42880643.40,192740725.29,1299321408.14,1674893735.07,http://www.iachina.cn/art/2018/3/6/art_617_102...,2017年保险业经营情况表,NaN,NaN,NaN
3,343975833.44,88130729.56,255845103.88,206441403.23,41055434.91,8348265.74,55361017.40,4157640.32,100908462.55,44522715.32,...,1994767.88,37058547.65,193021087.40,1277177694.44,1664096886.61,http://www.iachina.cn/art/2018/1/10/art_617_10...,2017年1-11月保险业经营情况表,NaN,NaN,NaN
4,323887679.96,79611362.01,244276317.95,198156253.23,38420882.44,7699182.28,50765401.89,3972071.35,90846751.60,39443910.98,...,1784671.48,33410032.12,191123741.21,1278381459.75,1661984320.20,http://www.iachina.cn/art/2017/11/27/art_617_9...,2017年1-10月保险业经营情况表,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,49504913.87,12089320.63,37415593.24,34211170.38,2252622.26,951800.60,NaN,NaN,13081729.46,5913100.94,...,248749.81,4404294.17,105293958.23,224736995.09,361956016.37,http://www.iachina.cn/art/2009/11/5/art_617_11...,2009年1-5月保险业经营数据,--,--,--
103,41404593.30,9827050.17,31577543.13,28972789.38,1827127.81,777625.94,NaN,NaN,10773711.41,4790724.18,...,202573.98,3524507.80,103943128.69,220258335.04,355455087.17,http://www.iachina.cn/art/2009/11/5/art_617_11...,2009年1-4月保险业经营数据,--,--,--
104,32767025.59,7185819.94,25581205.65,23589945.93,1403958.63,587301.08,NaN,NaN,8235800.22,3514472.15,...,151276.72,2617023.35,100047101.33,221784969.58,354347298.03,http://www.iachina.cn/art/2009/11/5/art_617_11...,2009年1-3月保险业经营数据,639228.12,5376276.26,4236221.70
105,32767025.59,7185819.94,25581205.65,23589945.93,1403958.63,587301.08,NaN,NaN,8235800.22,3514472.15,...,151276.72,2617023.35,100047101.33,221784969.58,354347298.03,http://www.iachina.cn/art/2009/11/5/art_617_11...,2009年1-2月保险业经营数据,639228.12,5376276.26,4236221.70
